In [126]:
import pandas as pd

In [127]:
game_results = pd.read_csv("game_results.csv")
player_stats = pd.read_csv("player_stats.csv")

C:\Users\whitehes\AppData\Local\Temp\ipykernel_16972\147419742.py:2: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  player_stats = pd.read_csv("player_stats.csv")


In [128]:
player_stats['season'] = [i.strftime('%Y') for i in pd.to_datetime(player_stats['utcStartTime']).to_list()]

In [129]:
cols_to_sum = ['goals', 'behinds', 'kicks', 'handballs', 'disposals', 'marks', 'bounces',
       'tackles', 'contestedPossessions', 'uncontestedPossessions',
       'totalPossessions', 'inside50s', 'marksInside50', 'contestedMarks',
       'hitouts', 'onePercenters', 'disposalEfficiency', 'clangers',
       'freesFor', 'freesAgainst', 'rebound50s',
       'goalAssists', 'turnovers', 'intercepts', 'tacklesInside50', 'shotsAtGoal',
       'scoreInvolvements', 'metresGained', 'clearances.centreClearances',
       'clearances.stoppageClearances', 'clearances.totalClearances',
       'extendedStats.effectiveKicks', 'extendedStats.kickToHandballRatio', 'extendedStats.effectiveDisposals',
       'extendedStats.marksOnLead', 'extendedStats.interceptMarks',
       'extendedStats.hitoutsToAdvantage', 'extendedStats.groundBallGets',
       'extendedStats.f50GroundBallGets', 'extendedStats.scoreLaunches',
       'extendedStats.pressureActs', 'extendedStats.defHalfPressureActs',
       'extendedStats.spoils', 'extendedStats.ruckContests',
       'extendedStats.contestDefOneOnOnes', 'extendedStats.contestDefLosses',
       'extendedStats.contestOffOneOnOnes', 'extendedStats.contestOffWins',
       'extendedStats.centreBounceAttendances', 'extendedStats.kickins',
       'extendedStats.kickinsPlayon']
group_by = ['season','round.roundNumber','team.name']
df_cols = cols_to_sum + group_by

In [130]:
subset_player_stats = player_stats[df_cols]

In [111]:
team_stats = subset_player_stats.groupby(group_by).sum()
team_stats.to_csv('team_stats.csv')

In [125]:
cols = ['year', 'round', 'team', 'score']
home_team = game_results[['round.year','round.roundNumber','match.homeTeam.name','homeTeamScore.matchScore.totalScore']]
home_team.columns = cols
away_team = game_results[['round.year','round.roundNumber','match.awayTeam.name','awayTeamScore.matchScore.totalScore']]
away_team.columns = cols
scores_df = home_team.append(away_team)
scores_df.reset_index(inplace=True)
scores_df.drop('index', axis=1, inplace=True)
scores_df.to_csv('team_score_results.csv')

ValueError: Length mismatch: Expected axis has 4 elements, new values have 5 elements

In [100]:
grouped_scores = scores_df.groupby(['year', 'round', 'team']).sum()

In [113]:
team_scores = pd.read_csv('team_score_results.csv')
team_stats = pd.read_csv('team_stats.csv')

In [116]:
combined_data_raw = team_stats.merge(team_scores, left_on=['season', 'round.roundNumber', 'team.name'], right_on=['year', 'round', 'team'])
combined_data = combined_data_raw.drop(['Unnamed: 0', 'year', 'round', 'team'], axis=1)

In [118]:
combined_data.to_csv('merged_stat_score_data_clean.csv',index=False)